In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

# Load dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight','Acceleration','Model Year','Origin']
raw_dataset = pd.read_csv(url, names=column_names, na_values='?', comment='\t', sep=' ', skipinitialspace=True)
dataset = raw_dataset.dropna()

# One-hot encode 'Origin'
features = dataset.drop(['MPG'], axis=1)
features = pd.get_dummies(features, columns=['Origin'], prefix='Origin')
labels = dataset['MPG']

# Normalize features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Train-test split
train_features, test_features, train_labels, test_labels = train_test_split(scaled_features, labels, test_size=0.2, random_state=42)

# Define model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[train_features.shape[1]]),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])

# Train model
model.fit(train_features, train_labels, epochs=100, validation_split=0.2, verbose=0)

# Evaluate
test_loss, test_mae, test_mse = model.evaluate(test_features, test_labels, verbose=0)
print(f"Test MAE: {test_mae:.2f}, Test MSE: {test_mse:.2f}")

# Interactive input
print("\nEnter the following details of the car:")
cylinders = float(input("Cylinders: "))
displacement = float(input("Displacement: "))
horsepower = float(input("Horsepower: "))
weight = float(input("Weight: "))
acceleration = float(input("Acceleration: "))
model_year = float(input("Model Year (e.g., 81): "))
origin = int(input("Origin (1: USA, 2: Europe, 3: Japan): "))

# Create input DataFrame
input_data = pd.DataFrame([[cylinders, displacement, horsepower, weight, acceleration, model_year, origin]],
                          columns=column_names[1:])

# One-hot encode 'Origin'
input_data = pd.get_dummies(input_data, columns=['Origin'], prefix='Origin')

# Add missing columns (if any)
for col in features.columns:
    if col not in input_data.columns:
        input_data[col] = 0
input_data = input_data[features.columns]  # Ensure column order

# Normalize input
scaled_input = scaler.transform(input_data)

# Predict
predicted_mpg = model.predict(scaled_input)
print(f"\n✅ Predicted MPG for the entered car: {predicted_mpg[0][0]:.2f}")
